In [ ]:
""" The Goal here is to Genarate a signal tabel. from the signal clmn
    which includes
    1. Genarate Sl
    2. Genarate Tp < hardCoded - Dinamic is comming in next version
 """

In [ ]:
""" IMPORTING all"""

import mplfinance as mpf
import numpy as np
import pandas as pd

from resampler import EnterpriseDataResampler
from tafm import create_analyzer, IndicatorConfig
from ChartterX5 import Chartter

In [ ]:
import pandas as pd

# Read the saved parquet file
ohlcv = pd.read_parquet("ohlcv-1M-mt5.parquet")


In [ ]:
ohlcv = ohlcv.rename(columns={
    "tickvol": "vol"  # fix spelling too
})
ohlcv.head(3)


In [ ]:
resampler = EnterpriseDataResampler()

dta = resampler.resample_data(
        resampler.filter_business_data(
            resampler.date_filter.filter_by_date_range(
                ohlcv,
                start_date="2024-10-01",
                end_date="2024-10-02"
            ),
            business_hours_only=False,
            weekdays_only=True
        ),
        period="1T"
    ).set_index("datetime")

In [ ]:
"""Applying tecNical Ananlisis.."""
dta = dta.rename(columns={
    "open": "Open",
    "high": "High",
    "low": "Low",
    "close": "Close",
    "vol": "Volume"  # fix spelling too
})

analyzer = create_analyzer(dta)

analyzer.add_indicator(IndicatorConfig(name='RSI', period=14, source_column='Close'))
analyzer.add_indicator(IndicatorConfig(name='EMA', period=15, source_column='RSI_14'))


In [ ]:
#>=Signal Genarator

analyzer.crossed_up('RSI_14', 'EMA_15')

analyzer.df.tail(10)

In [ ]:
ohlcv = analyzer.df.rename(columns={
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume"  # fix spelling too
})


In [ ]:
# Initialize chartter with wider and less tall proportions, no volume
chartter = Chartter(config={
    'chart_type': 'candle',
    'style': 'charles',
    'figratio': (20, 8),  # Adjusted for wider and less tall chart
    'volume': False  # Disable volume bars
})

# --- TradingView Market Colors ---
tv_mc = mpf.make_marketcolors(
    up='#26a69a',         # TradingView teal green for up candles
    down='#ef5350',       # TradingView red for down candles
    edge='inherit',       # Clean edges matching candle color
    wick='inherit',       # Wicks match candle colors
)

# --- TradingView Dark Theme ---
tradingview_dark = mpf.make_mpf_style(
    base_mpf_style='nightclouds',  # Start with dark base
    marketcolors=tv_mc,

    # TradingView dark theme colors
    figcolor='#131722',           # Dark navy background
    facecolor='#1e222d',          # Dark gray chart area
    gridcolor='#363a45',          # Dark gray grid
    gridstyle='-',                # Solid grid lines

    y_on_right=True,              # Price axis on right

    rc={
        # TradingView typography
        # 'font.family': ['-apple-system', 'BlinkMacSystemFont', 'Trebuchet MS', 'Roboto', 'Ubuntu', 'sans-serif'],
        'axes.labelsize': 10,
        'axes.titlesize': 14,
        'xtick.labelsize': 9,
        'ytick.labelsize': 7,         # Reduced size for price tick labels
        'legend.fontsize': 9,

        # Clean spacing with reduced left padding and full-width chart
        'axes.labelpad': 10,
        'xtick.major.pad': 6,
        'ytick.major.pad': 6,
        'axes.xmargin': 0,            # Remove left/right margins
        'axes.ymargin': 0,            # Remove top/bottom margins
        'figure.subplot.left': 0.05,  # Minimize left subplot padding
        'figure.subplot.right': 0.95, # Maximize right subplot to fit price scale

        # TradingView-style lines
        'lines.linewidth': 1.5,
        'lines.antialiased': True,

        # Clean borders and colors
        'axes.edgecolor': '#434651',  # Darker edge for dark theme
        'axes.linewidth': 1,
        'xtick.color': '#787b86',     # Light gray for x-axis ticks
        'ytick.color': '#787b86',     # Light gray for y-axis ticks
        'axes.labelcolor': 'none',    # Hide axis label color (labels are empty)

        # Grid styling
        'axes.grid': True,
        'axes.axisbelow': True,
        'grid.alpha': 0.6,            # Slightly more transparent for dark theme
        'grid.linewidth': 0.8,

        # Clean spines (show right spine for price scale)
        'axes.spines.left': True,
        'axes.spines.bottom': True,
        'axes.spines.top': False,
        'axes.spines.right': True,    # Show right spine for price scale
        'ytick.right': True,          # Show right y-axis ticks
    }
)

#tradingview_config_4H
tradingview_config_4H = {
    'title': 'XAUUSD, 4H',                # TradingView style title
    'style': tradingview_dark,             # Use dark theme
    'type': 'candle',
    'volume': False,                      # Disable volume bars
    'show_nontrading': False,             # Ensure no extra y-axis spacing
    'datetime_format': '%Y-%m-%d',        # Format x-axis to show date only
    'xlabel': '',                         # Remove x-axis label
    'ylabel': '',                         # Remove y-axis label
    'xrotation': 0                        # Horizontal date labels
}


In [ ]:
dpart = ohlcv[:300]

fig, axes = chartter.plot(
    dpart,
    addplot = [
        mpf.make_addplot(dpart['RSI_14'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(dpart['EMA_15'], panel=1, color='blue', ylabel='ema15'),
    ],
    config=tradingview_config_4H,
    vlines=dict(
        vlines=dpart.index[dpart['RSI_14_crossed_up_EMA_15'] == 1].to_list(),
        colors='white',
        linewidths=0.3,
        alpha=0.1
    ),
    returnfig=True
)

In [ ]:
import matplotlib.pyplot as plt
import mplfinance as mpf

dpart = ohlcv[:300]

# First, let's check what columns we have
print("Available columns:", dpart.columns.tolist())
print("DataFrame shape:", dpart.shape)
print("First few rows:")
print(dpart.head())

# Common column name variations for OHLCV data
possible_names = {
    'open': ['Open', 'open', 'OPEN', 'o'],
    'high': ['High', 'high', 'HIGH', 'h'],
    'low': ['Low', 'low', 'LOW', 'l'],
    'close': ['Close', 'close', 'CLOSE', 'c'],
    'volume': ['Volume', 'volume', 'VOLUME', 'v']
}

# Find the correct column names
col_map = {}
for ohlcv_name, variations in possible_names.items():
    for variation in variations:
        if variation in dpart.columns:
            col_map[ohlcv_name] = variation
            break

print("Detected column mapping:", col_map)

# Create subplots
fig, (ax_candle, ax_rsi) = plt.subplots(2, 1, figsize=(14, 10), sharex=True,
                                       gridspec_kw={'height_ratios': [3, 2]})

# Only proceed if we have the required OHLC columns
if all(key in col_map for key in ['open', 'high', 'low', 'close']):
    # Plot candlestick manually
    for idx, (timestamp, row) in enumerate(dpart.iterrows()):
        # Get OHLC values using the correct column names
        open_val = row[col_map['open']]
        high_val = row[col_map['high']]
        low_val = row[col_map['low']]
        close_val = row[col_map['close']]

        # Determine color
        color = 'green' if close_val > open_val else 'red'

        # Draw high-low line (wick)
        ax_candle.plot([idx, idx], [low_val, high_val], color='black', linewidth=0.8)

        # Draw open-close rectangle (body)
        height = abs(close_val - open_val)
        bottom = min(open_val, close_val)
        rect = plt.Rectangle((idx-0.4, bottom), 0.8, height, facecolor=color, alpha=0.8, edgecolor='black', linewidth=0.5)
        ax_candle.add_patch(rect)

    # Set x-axis labels to show actual timestamps at regular intervals
    n_ticks = 10
    tick_indices = list(range(0, len(dpart), max(1, len(dpart)//n_ticks)))
    # ax_candle.set_xticks(tick_indices)
    ax_candle.set_xticklabels([dpart.index[i].strftime('%m-%d %H:%M') for i in tick_indices], rotation=45)

    ax_candle.set_title('Candlestick Chart', fontsize=14)
    ax_candle.set_ylabel('Price', fontsize=12)
    ax_candle.grid(True, alpha=0.3)
else:
    ax_candle.text(0.5, 0.5, 'OHLC columns not found!\nAvailable columns: ' + str(list(dpart.columns)),
                   transform=ax_candle.transAxes, ha='center', va='center')

# Plot RSI and EMA on the second subplot (check if these columns exist)
if 'RSI_14' in dpart.columns and 'EMA_15' in dpart.columns:
    ax_rsi.plot(range(len(dpart)), dpart['RSI_14'], color='purple', label='RSI_14', linewidth=2)
    ax_rsi.plot(range(len(dpart)), dpart['EMA_15'], color='blue', label='EMA_15', linewidth=2)

    # Add vertical lines for crosses (if the column exists)
    if 'RSI_14_crossed_up_EMA_15' in dpart.columns:
        cross_mask = dpart['RSI_14_crossed_up_EMA_15'] == 1
        cross_indices = [i for i, val in enumerate(cross_mask) if val]
        for idx in cross_indices:
            ax_candle.axvline(x=idx, color='red', linewidth=1, alpha=0.7)
            ax_rsi.axvline(x=idx, color='red', linewidth=1, alpha=0.7)

    # Customize RSI chart
    ax_rsi.set_ylabel('RSI / EMA Values', fontsize=12)
    ax_rsi.set_xlabel('Time', fontsize=12)
    ax_rsi.legend(fontsize=10)
    ax_rsi.grid(True, alpha=0.3)
    ax_rsi.set_title('RSI and EMA Indicators', fontsize=12)

    # Share x-axis formatting
    if tick_indices:
        ax_rsi.set_xticks(tick_indices)
        ax_rsi.set_xticklabels([dpart.index[i].strftime('%m-%d %H:%M') for i in tick_indices], rotation=45)
else:
    ax_rsi.text(0.5, 0.5, 'RSI_14 and/or EMA_15 columns not found!\nAvailable columns: ' + str(list(dpart.columns)),
                transform=ax_rsi.transAxes, ha='center', va='center')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
dta = dpart.copy()


dta['SL'] = dta['low'] - (dta ['high'] - dta ['low'])


dta['TP'] = dta['close'] + (dta ['high'] - dta ['low']) * 2


dta.head(3)

In [ ]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.patches as mpatches

# Get row numbers (integer positions) where signal is 1
row_numbers = np.where(dta['RSI_14_crossed_up_EMA_15'] == 1)[0]

# Extract SL and TP values for those rows
signal = dta[dta['RSI_14_crossed_up_EMA_15'] == 1]
sl_array = signal['close'].to_numpy()
sl_array = signal['SL'].to_numpy()
tp_array = signal['TP'].to_numpy()

# Optional: combine into structured array
signal_array = np.array(list(zip(row_numbers, sl_array, tp_array)),
                        dtype=[('row', int), ('SL', float), ('TP', float)])

signal_array

In [ ]:
# -----------------------------
# 3. Define function to draw rectangles
# -----------------------------
def draw_signal_boxes(ax, signal_array):
    for entry in signal_array:
        row = entry['row']
        sl = entry['SL']
        tp = entry['TP']
        rect = mpatches.Rectangle(
            (row - 0.5, sl),   # bottom-left corner: x=row, y=SL
            1,                 # width = 1 candle
            tp - sl,           # height = TP - SL
            color='red',
            alpha=0.3
        )
        ax.add_patch(rect)

# axes[0] is the main price axes
draw_signal_boxes(axes[0], signal_array)

fig


In [ ]:
dta

In [ ]:
# Save DataFrame as Parquet
dta.to_parquet("sampleSignal.parquet", index=True)  # index=True to keep datetime index
